<div style="background-color: #eef5fc; padding: 20px; border: 1px solid #b8d2eb; border-radius: 10px; margin: 10px 0;">

# 📈 Empirical Analysis of Market Capitalization Weighted Index (MWI) vs. Equal Weighted Index (EWI) Performance for S&P 500

<p style="font-size: 18px;">This notebook presents an empirical analysis of the performance of Market Capitalization Weighted Indexing (MWI) and Equal Weighted Indexing (EWI) strategies applied to a subset of the S&P 500. The objective is to explore why the EWI strategy often outperforms the MWI strategy.</p>

<p style="font-size: 18px;">For the sake of stability and simplicity in this analysis, we are working with the 411 companies that have continuously been part of the S&P 500 from January 2006 to August 2023. This eliminates any effects due to companies entering or leaving the index during this period.</p>

<p style="font-size: 18px;">We begin by fetching the necessary data for these companies using open-source Python package like `yfinance` and web scraping techniques. All the data used in this analysis is freely available and no proprietary or paid sources were used.</p>

<p style="font-size: 18px;">Next, we build MWI and EWI portfolios from this data, calculating returns and other relevant metrics. Please note that we do not consider transaction fees in these custom portfolios to avoid additional complexity. Furthermore, for the rebalancing of both portfolios, we perform it after the close of the third Friday in March, June, September, and December.</p>

<p style="font-size: 18px;">Through exploratory data analysis, we delve into the distribution characteristics of individual stock returns, focusing on their skewness as a potential explanatory factor for the observed performance difference. We also visualize various aspects of our portfolios to better understand their composition and performance over time.</p>

<p style="font-size: 20px; font-weight: bold; text-align: center;">Let's begin!</p>

<img src="stock.jpg" alt="Image Description" style="max-width:100%; height:auto;">

</div>


---

<div style="background-color: #eef5fc; padding: 20px; border: 1px solid #b8d2eb; border-radius: 5px;">

## 📈 Step 1: Fetching the Tickers of S&P 500 Constituents

<p style="font-size: 16px;">The function <code>get_sp500_tickers()</code> fetches the ticker symbols of all companies that are currently constituents of the S&P 500 index.</p>

<p style="font-size: 16px;">This is done by web scraping the Wikipedia page that lists the S&P 500 constituents. We are using the pandas <code>read_html</code> function which conveniently fetches tabular data from an HTML page and returns it in the form of a DataFrame. After fetching the data, we just extract the ticker symbols and return them as a list.</p>

</div>

---


In [75]:
import pandas as pd 

def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    table = pd.read_html(url, header=0)
    df = table[0]
    return list(df.Symbol)

<div style="background-color: #ffffcc; padding: 10px; border: 1px solid #ffcc00; border-radius: 5px; margin: 10px 0;">
    <span style="color: #666699; font-style: italic;">🔔</span> <span style="color:blue; font-weight: bold;">After running this function, we will have a list of ticker symbols for all current S&amp;P 500 constituents stored in the `tickers` variable.</span>
</div>


In [4]:
tickers = get_sp500_tickers()

In [10]:
sorted(tickers)[:10]

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACGL', 'ACN', 'ADBE']

---

<div style="background-color: #eef5fc; padding: 20px; border: 1px solid #b8d2eb; border-radius: 5px;">

## 📊 Step 2: Fetching Historical Market Capitalization Data 

<p style="font-size: 16px;">To construct our <strong>Market Capitalization Weighted Index (MWI)</strong>, we need to collect historical daily market capitalization data for each constituent of the S&P 500. Sourcing this type of data can often be challenging due to access restrictions and cost constraints. After thorough exploration, I found a freely available API endpoint on StockAnalysis.com that supplies this precise information.</p>

<p style="font-size: 16px;">This endpoint was discovered by examining the Network tab in the browser's Developer Tools while interacting with the <code>stockanalysis.com</code> website. This allowed us to observe the API calls made by the web application.</p>

<p style="font-size: 16px;">The API endpoint we identified is <code>https://stockanalysis.com/api/symbol/s/{symbol}/marketcap?t=price</code>, where <code>{symbol}</code> is the ticker symbol of a stock we're interested in. This API call returns a JSON object with a <code>status</code> field, indicating the success or failure of the request, and a <code>data</code> field containing the actual market cap data. The <code>data</code> field is an array of arrays, where each inner array consists of a Unix timestamp (representing the date) and the market cap for the specified stock on that date.</p>

<p style="font-size: 16px;">To effectively gather data for all the stocks in our list, I implemented a concurrent fetching system using the <code>concurrent.futures</code> module. This system allows us to send multiple API requests simultaneously, significantly reducing the total time required to gather all our necessary data.</p>

<p style="font-size: 16px;">In the following code, we will pull historical market capitalization data for all currently listed stocks in the S&P 500 index.</p>

</div>

---


In [11]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tenacity import retry, stop_after_attempt, wait_fixed

@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def get_market_cap(symbol):
    url = f"https://stockanalysis.com/api/symbol/s/{symbol}/marketcap?t=price"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame({
        'Date': [pd.to_datetime(x[0], unit='ms') for x in data['data']],
        f'Market Cap {symbol}': [x[1] for x in data['data']]
    })

def get_market_caps(tickers):
    first_ticker = tickers.pop(0)
    master_df = get_market_cap(first_ticker)

    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_ticker = {executor.submit(get_market_cap, ticker): ticker for ticker in tickers}
        for future in as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                df = future.result()
                master_df = master_df.merge(df, on='Date', how='outer')
            except Exception as e:
                print(f"Error occurred while getting market cap data for {ticker}: {e}")

    return master_df

In [12]:
data = get_market_caps(tickers)

In [42]:
data.set_index('Date', inplace=True)

In [43]:
data.head()

,Market Cap MMM,Market Cap ABBV,Market Cap AOS,Market Cap ABT,Market Cap ACN,Market Cap ATVI,Market Cap ADM,Market Cap ADBE,Market Cap ADP,Market Cap AAP,...,Market Cap WTW,Market Cap GWW,Market Cap WYNN,Market Cap XEL,Market Cap XYL,Market Cap ZBRA,Market Cap ZBH,Market Cap YUM,Market Cap ZION,Market Cap ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1998-12-01,3.266150e+10,NaN,610000000.0,7.266370e+10,NaN,NaN,NaN,3.018500e+09,2.330110e+10,NaN,...,NaN,4.106500e+09,NaN,NaN,NaN,1.046800e+09,NaN,7.022000e+09,4.022400e+09,NaN
1998-12-02,3.248490e+10,NaN,612800000.0,7.418130e+10,NaN,NaN,NaN,2.957000e+09,2.398090e+10,NaN,...,NaN,4.095200e+09,NaN,NaN,NaN,1.044900e+09,NaN,6.936400e+09,4.100600e+09,NaN
1998-12-03,3.188290e+10,NaN,611400000.0,7.266370e+10,NaN,NaN,NaN,3.018500e+09,2.284800e+10,NaN,...,NaN,4.112200e+09,NaN,NaN,NaN,1.039000e+09,NaN,7.098400e+09,4.171000e+09,NaN
1998-12-04,3.220800e+10,NaN,612800000.0,7.550170e+10,NaN,NaN,NaN,2.965400e+09,2.339480e+10,NaN,...,NaN,4.207200e+09,NaN,NaN,NaN,1.039000e+09,NaN,7.537400e+09,4.257000e+09,NaN
1998-12-07,3.092770e+10,NaN,598400000.0,7.550170e+10,NaN,NaN,NaN,3.064200e+09,2.358210e+10,NaN,...,NaN,4.142600e+09,NaN,NaN,NaN,1.062400e+09,NaN,7.547800e+09,4.261700e+09,NaN


---

<div style="background-color: #eef5fc; padding: 20px; border: 1px solid #b8d2eb; border-radius: 5px;">

## 🧹 Step 3: Data Cleaning

<p style="font-size: 16px;"><span style="color:blue; font-style: italic;">Insight:</span> In this phase, our primary goal is to scrutinize and tidy up our dataset. By analyzing the count of non-NaN values for each year, we gain insight into the number of companies present in our dataset for each respective year.</p>

</div>

---


In [48]:
import numpy as np
non_zero_counts = data.groupby(data.index.year).count().replace(0, np.nan).apply(lambda x: x.notnull().sum(), axis=1)
non_zero_counts

Date
1983      2
1984      2
1985      2
1986      2
1987      3
1988      3
1989      3
1990      3
1991      3
1992      3
1993      3
1994      3
1995      3
1996      3
1997      3
1998    334
1999    349
2000    363
2001    384
2002    388
2003    393
2004    405
2005    412
2006    418
2007    427
2008    431
2009    435
2010    444
2011    450
2012    457
2013    468
2014    476
2015    483
2016    485
2017    489
2018    494
2019    498
2020    500
2021    501
2022    502
2023    503
dtype: int64

<div style="background-color: #ffffcc; padding: 10px; border: 1px solid #ffcc00; border-radius: 5px; margin: 10px 0;">
    <span style="color: #666699; font-style: italic;">🔔</span> <span style="color:blue; font-weight: bold;">:</span> <span style="color:blue; font-style: italic;">Starting from 2013 offers a comprehensive perspective because it provides over a decade of data coverage and represents more than 95% of the current S&P 500 companies.</span>
</div>


In [57]:
data_2013 = data[data.index.year >= 2013]

df = data_2013.dropna(axis=1)

df.head()

,Market Cap MMM,Market Cap ABBV,Market Cap AOS,Market Cap ABT,Market Cap ACN,Market Cap ATVI,Market Cap ADM,Market Cap ADBE,Market Cap ADP,Market Cap AAP,...,Market Cap WHR,Market Cap WTW,Market Cap GWW,Market Cap WYNN,Market Cap XEL,Market Cap XYL,Market Cap ZBRA,Market Cap ZBH,Market Cap YUM,Market Cap ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,6.558120e+10,5.548960e+10,2.982800e+09,5.066040e+10,5.487730e+10,1.217860e+10,1.888110e+10,1.898040e+10,2.834680e+10,5.318200e+09,...,8.343600e+09,5.924800e+09,1.443610e+10,1.187570e+10,1.335100e+10,5.109300e+09,2.083100e+09,1.174610e+10,3.034350e+10,4.049900e+09
2013-01-03,6.550510e+10,5.503140e+10,2.936300e+09,5.258880e+10,5.467860e+10,1.225090e+10,1.872960e+10,1.868830e+10,2.845850e+10,5.318200e+09,...,8.322600e+09,5.985300e+09,1.439020e+10,1.200340e+10,1.331200e+10,5.092600e+09,2.085200e+09,1.191440e+10,3.053320e+10,4.055400e+09
2013-01-04,6.598950e+10,5.433780e+10,2.968300e+09,5.227270e+10,5.498060e+10,1.242890e+10,1.924330e+10,1.887640e+10,2.871000e+10,5.401100e+09,...,8.251700e+09,5.983600e+09,1.448410e+10,1.214820e+10,1.336080e+10,5.085200e+09,2.068500e+09,1.197510e+10,3.086760e+10,4.171400e+09
2013-01-07,6.607250e+10,5.444680e+10,2.935400e+09,5.269950e+10,5.474220e+10,1.244000e+10,1.844640e+10,1.878240e+10,2.859930e+10,5.382700e+09,...,8.125600e+09,5.966300e+09,1.439370e+10,1.218950e+10,1.321940e+10,5.018300e+09,2.080100e+09,1.200460e+10,3.067330e+10,4.151200e+09
2013-01-08,6.607940e+10,5.326180e+10,2.991300e+09,5.271530e+10,5.506000e+10,1.222860e+10,1.866380e+10,1.888140e+10,2.876430e+10,5.294700e+09,...,8.175400e+09,5.936800e+09,1.415330e+10,1.217890e+10,1.324370e+10,4.957000e+09,2.081600e+09,1.202200e+10,2.938560e+10,4.068300e+09


<div style="background-color: #ffffcc; padding: 10px; border: 1px solid #ffcc00; border-radius: 5px; margin: 10px 0;">
    <span style="color: #666699; font-style: italic;">🔔</span> <span style="color:blue; font-weight: bold;">Note:</span> <span style="color:blue; font-style: italic;">There's a discrepancy in the number of companies reported for 2013 pre (468) and post data cleaning (458). This is due to Partial Data. During our initial aggregation, companies with minimal data points for 2013 were included. However, post-cleaning, companies lacking substantial data coverage for the entire year were filtered out, leading to the reduced count. This ensures our dataset only includes companies with data uniformly spread over the years.</span>
</div>


---

<div style="background-color: #eef5fc; padding: 20px; border: 1px solid #b8d2eb; border-radius: 5px;">

### 💹 Step 4: Yahoo Finance Data Integration

<p style="font-size: 16px;">Here's what we're doing in this step:</p>

1. **Ticker Cleaning**: 
   <div style="margin-left: 20px">Ensuring tickers are Yahoo Finance compatible.</div>
   
2. **Ticker Conversion**: 
   <div style="margin-left: 20px">Extracting and converting tickers from the dataset's columns for querying.</div>

3. **Date Range**: 
   <div style="margin-left: 20px">Establishing the date range using the min and max dates from the dataset.</div>

4. **Yahoo Finance Query**: 
   <div style="margin-left: 20px">Fetching price data for our tickers within the specified date range using `yfinance`.</div>

5. **DataFrame Prep**: 
   <div style="margin-left: 20px">Adjusting the column structure of our dataset for the integration with the Yahoo Finance data.</div>

6. **Data Merge**: 
   <div style="margin-left: 20px">Concatenating the Yahoo Finance data with our market data dataset.</div>

</div>

---


In [76]:
import yfinance as yf


tickers = [col.replace('Market Cap ', '') for col in df.columns]
tickers = [ticker.replace("BF.B", "BF-B").replace("BRK.B", "BRK-B") for ticker in tickers]
tickers_str = ' '.join(tickers)

start_date = df.index.min()
end_date = df.index.max()

yf_data = yf.download(tickers_str, start=start_date, end=end_date, group_by='ticker')

df_copy = df.copy()
df_copy.columns = pd.MultiIndex.from_product([tickers, ['Market Cap']])


merged_data = pd.concat([yf_data, df_copy], axis=1)


[*********************100%***********************]  458 of 458 completed


In [77]:
merged_data = merged_data.sort_index(axis=1)

In [78]:
merged_data.head()

A                                                 \
            Adj Close      Close       High        Low    Market Cap   
Date                                                                   
2013-01-02  27.338417  29.957083  30.164520  29.434908  1.457150e+10   
2013-01-03  27.436333  30.064379  30.143063  29.742489  1.462370e+10   
2013-01-04  27.978144  30.658083  30.722462  29.942776  1.491250e+10   
2013-01-07  27.775774  30.436338  30.615166  30.314735  1.480460e+10   
2013-01-08  27.553833  30.193132  30.600859  30.114449  1.468630e+10   

                                        AAP                        ...  \
                 Open     Volume  Adj Close      Close       High  ...   
Date                                                               ...   
2013-01-02  30.164520  8790205.0  66.418594  72.489998  73.760002  ...   
2013-01-03  30.000000  5751791.0  66.418594  72.489998  73.440002  ...   
2013-01-04  30.143063  6432897.0  67.453972  73.620003  73.639999  ...   
2013-01-07  30.472103  3589505.0  67.224915  73.370003  73.800003  ...   
2013-01-08  30.379112  3896925.0  66.125397  72.169998  73.660004  ...   

                    ZBRA                            ZION             \
              Market Cap       Open    Volume  Adj Close      Close   
Date                                                                  
2013-01-02  2.083100e+09  40.040001  213800.0  18.007875  21.990000   
2013-01-03  2.085200e+09  40.790001   99900.0  18.032444  22.020000   
2013-01-04  2.068500e+09  40.509998  146000.0  18.548361  22.650000   
2013-01-07  2.080100e+09  40.570000   90600.0  18.458277  22.540001   
2013-01-08  2.081600e+09  40.650002  112600.0  18.089771  22.090000   

                                                                      
                 High        Low    Market Cap       Open     Volume  
Date                                                                  
2013-01-02  22.000000  21.780001  4.049900e+09  21.879999  2551100.0  
2013-01-03  22.129999  21.809999  4.055400e+09  21.990000  2267900.0  
2013-01-04  22.790001  21.969999  4.171400e+09  22.020000  3577700.0  
2013-01-07  22.580000  22.260000  4.151200e+09  22.520000  2286000.0  
2013-01-08  22.530001  22.070000  4.068300e+09  22.500000  3758600.0  

[5 rows x 3206 columns]

---

<div style="background-color: #eef5fc; padding: 20px; border: 1px solid #b8d2eb; border-radius: 5px;">

### 📊 Step 5: Designing Equally-Weighted and Market-Capitalized ETF Portfolios

<p style="font-size: 16px;">This step delves into the construction of two distinct ETF portfolio strategies: <strong>EWI</strong> (Equal Weighted Index) and <strong>MWI</strong> (Market Weighted Index). The meticulous planning of these strategies demands various stages, including the computation of initial weights, the establishment of a rebalancing calendar, calculation of daily returns, and the culminating evaluation of the portfolio’s cumulative returns.</p>

1. **Initial Weights Assignment**: 
   <div style="margin-left: 20px">Setting up the portfolio weights for our two strategies: EWI and MWI.</div>
   
2. **Rebalancing Calendar**: 
   <div style="margin-left: 20px">Identifying specific dates, notably the third Friday of March, June, September, and December, as our strategic rebalancing points.</div>

3. **Rebalancing Logic**: 
   <div style="margin-left: 20px">Adjusting the weights of the assets in our portfolio based on the strategies. EWI is uniformly re-adjusted, while MWI is recalibrated according to the latest market caps.</div>

4. **Daily Return Calculation**: 
   <div style="margin-left: 20px">Computing daily returns for every ticker, based on their closing prices.</div>

5. **Portfolio Returns Computation**: 
   <div style="margin-left: 20px">Aggregating the ticker returns into portfolio returns for both EWI and MWI strategies.</div>

6. **Cumulative Return Analysis**: 
   <div style="margin-left: 20px">Assessing the overall performance of our strategies over time, by observing its growth.</div>

</div>

---


In [80]:
import pandas as pd
import numpy as np

data_copy = merged_data.copy()
tickers = data_copy.columns.get_level_values(0).unique()

weights_ewi = pd.DataFrame(index=data_copy.index, columns=tickers)
weights_mwi = pd.DataFrame(index=data_copy.index, columns=tickers)

weights_ewi.iloc[0] = 1 / len(tickers)
total_market_cap = data_copy.iloc[0].xs('Market Cap', level=1).sum()
weights_mwi.iloc[0] = data_copy.iloc[0].xs('Market Cap', level=1) / total_market_cap

third_fridays = pd.date_range(start=data_copy.index.min(), end=data_copy.index.max(), freq='WOM-3FRI')
rebalance_dates = third_fridays[third_fridays.month.isin([3, 6, 9, 12])]
rebalance_dates = rebalance_dates.intersection(data_copy.index)

for date in rebalance_dates:
    if date > weights_ewi.index.min():
        weights_ewi.loc[date] = 1 / len(tickers)
        market_caps = data_copy.loc[date].xs('Market Cap', level=1)
        total_market_cap = market_caps.sum()
        weights_mwi.loc[date] = market_caps / total_market_cap

weights_ewi = weights_ewi.fillna(method='ffill')
weights_mwi = weights_mwi.fillna(method='ffill')

for ticker in tickers:
    data_copy[(ticker, 'Return')] = data_copy[(ticker, 'Close')].pct_change()

data_copy[('EWI', 'Return')] = (data_copy.xs('Return', axis=1, level=1) * weights_ewi).sum(axis=1)
data_copy[('MWI', 'Return')] = (data_copy.xs('Return', axis=1, level=1) * weights_mwi).sum(axis=1)
data_copy[('EWI', 'Cumulative Return')] = (1 + data_copy[('EWI', 'Return')]).cumprod() - 1
data_copy[('MWI', 'Cumulative Return')] = (1 + data_copy[('MWI', 'Return')]).cumprod() - 1


C:\Users\Zahra\AppData\Local\Temp\ipykernel_9684\369957534.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_copy[(ticker, 'Return')] = data_copy[(ticker, 'Close')].pct_change()
C:\Users\Zahra\AppData\Local\Temp\ipykernel_9684\369957534.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_copy[('EWI', 'Return')] = (data_copy.xs('Return', axis=1, level=1) * weights_ewi).sum(axis=1)
C:\Users\Zahra\AppData\Local\Temp\ipykernel_9684\369957534.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

In [85]:
data_copy.head()

A                                                 \
            Adj Close      Close       High        Low    Market Cap   
Date                                                                   
2013-01-02  27.338417  29.957083  30.164520  29.434908  1.457150e+10   
2013-01-03  27.436333  30.064379  30.143063  29.742489  1.462370e+10   
2013-01-04  27.978144  30.658083  30.722462  29.942776  1.491250e+10   
2013-01-07  27.775774  30.436338  30.615166  30.314735  1.480460e+10   
2013-01-08  27.553833  30.193132  30.600859  30.114449  1.468630e+10   

                                        AAP                        ...  \
                 Open     Volume  Adj Close      Close       High  ...   
Date                                                               ...   
2013-01-02  30.164520  8790205.0  66.418594  72.489998  73.760002  ...   
2013-01-03  30.000000  5751791.0  66.418594  72.489998  73.440002  ...   
2013-01-04  30.143063  6432897.0  67.453972  73.620003  73.639999  ...   
2013-01-07  30.472103  3589505.0  67.224915  73.370003  73.800003  ...   
2013-01-08  30.379112  3896925.0  66.125397  72.169998  73.660004  ...   

                XRAY       XYL       YUM       ZBH      ZBRA      ZION  \
              Return    Return    Return    Return    Return    Return   
Date                                                                     
2013-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2013-01-03 -0.002715 -0.003273  0.006254  0.014328  0.000977  0.001364   
2013-01-04  0.008661 -0.001459  0.010950  0.005097 -0.008049  0.028610   
2013-01-07  0.003435 -0.013153 -0.006294  0.002463  0.005655 -0.004856   
2013-01-08 -0.011491 -0.012218 -0.041980  0.001445  0.000733 -0.019965   

                 EWI       MWI               EWI               MWI  
              Return    Return Cumulative Return Cumulative Return  
Date                                                                
2013-01-02  0.000000  0.000000          0.000000          0.000000  
2013-01-03 -0.000483 -0.001882         -0.000483         -0.001882  
2013-01-04  0.007239  0.005028          0.006752          0.003137  
2013-01-07 -0.003262 -0.002891          0.003469          0.000237  
2013-01-08 -0.003614 -0.003067         -0.000158         -0.002831  

[5 rows x 3668 columns]

<div style="background-color: #ffffcc; padding: 10px; border: 1px solid #ffcc00; border-radius: 5px; margin: 10px 0;">
    <span style="color: #666699; font-style: italic;">🔔</span> <span style="color:blue; font-weight: bold;">Drop the first row to remove NaN values</span>
</div>


In [87]:
df_portfolio = data_copy.iloc[1:]
df_portfolio = df_portfolio.sort_index(axis=1)
df_portfolio.head()

A                                                 \
            Adj Close      Close       High        Low    Market Cap   
Date                                                                   
2013-01-03  27.436333  30.064379  30.143063  29.742489  1.462370e+10   
2013-01-04  27.978144  30.658083  30.722462  29.942776  1.491250e+10   
2013-01-07  27.775774  30.436338  30.615166  30.314735  1.480460e+10   
2013-01-08  27.553833  30.193132  30.600859  30.114449  1.468630e+10   
2013-01-09  28.298004  31.008583  31.080114  30.536482  1.508300e+10   

                                                  AAP             ...  \
                 Open    Return     Volume  Adj Close      Close  ...   
Date                                                              ...   
2013-01-03  30.000000  0.003582  5751791.0  66.418594  72.489998  ...   
2013-01-04  30.143063  0.019748  6432897.0  67.453972  73.620003  ...   
2013-01-07  30.472103 -0.007233  3589505.0  67.224915  73.370003  ...   
2013-01-08  30.379112 -0.007991  3896925.0  66.125397  72.169998  ...   
2013-01-09  30.758226  0.027008  5737951.0  66.326965  72.389999  ...   

                ZBRA                 ZION                                   \
              Return    Volume  Adj Close      Close       High        Low   
Date                                                                         
2013-01-03  0.000977   99900.0  18.032444  22.020000  22.129999  21.809999   
2013-01-04 -0.008049  146000.0  18.548361  22.650000  22.790001  21.969999   
2013-01-07  0.005655   90600.0  18.458277  22.540001  22.580000  22.260000   
2013-01-08  0.000733  112600.0  18.089771  22.090000  22.530001  22.070000   
2013-01-09  0.010750   68900.0  17.893230  21.850000  22.160000  21.719999   

                                                      
              Market Cap   Open    Return     Volume  
Date                                                  
2013-01-03  4.055400e+09  21.99  0.001364  2267900.0  
2013-01-04  4.171400e+09  22.02  0.028610  3577700.0  
2013-01-07  4.151200e+09  22.52 -0.004856  2286000.0  
2013-01-08  4.068300e+09  22.50 -0.019965  3758600.0  
2013-01-09  4.024100e+09  22.02 -0.010865  3031000.0  

[5 rows x 3668 columns]

---

<div style="background-color: #eef5fc; padding: 20px; border: 1px solid #b8d2eb; border-radius: 5px;">

### 📊 Step 6: Visualization of Cumulative Returns MWI vs EWI

<p style="font-size: 16px;">In this step, we employ an interactive visualization to contrast the performances of the two portfolio strategies: <strong>EWI</strong> (Equal Weighted Index) and <strong>MWI</strong> (Market Weighted Index).</p>

<p style="color: blue; font-size: 16px; font-style: italic;">Insight:</p>
<p style="font-size: 16px;">A preliminary observation from the chart showcases that the cumulative return of EWI outpaces that of MWI over the studied period almost all the time. This suggests that the equal allocation strategy (where each asset holds equal weight) has outperformed the market-cap-based one.</p>

</div>

---


In [89]:
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(x=df_portfolio.index, y=df_portfolio[('EWI', 'Cumulative Return')],
                    mode='lines',
                    name='EWI'))

fig.add_trace(go.Scatter(x=df_portfolio.index, y=df_portfolio[('MWI', 'Cumulative Return')],
                    mode='lines',
                    name='MWI'))


fig.update_layout(title='Cumulative Returns of EWI and MWI Portfolios',
                   xaxis_title='Date',
                   yaxis_title='Cumulative Return',
                   template="plotly_dark") 


fig.show()

<div style="background-color: #eef5fc; padding: 20px; border: 1px solid #b8d2eb; border-radius: 5px;">

### 📊 Step 7: Dissecting Weight Distributions: MWI vs EWI

<p style="font-size: 16px;">In this step, we delve into the weight distribution characteristics of the two indexing strategies: <strong>MWI</strong> (Market-Cap Weighted Index) and <strong>EWI</strong> (Equal Weighted Index).</p>

<p style="color: blue; font-size: 16px; font-style: italic;">Insight:</p>
<p style="font-size: 16px;">By understanding the disparity in weight allocation across stocks in these two strategies, we can gain deeper insights into why MWI lag behind EWI.</p>

</div>
